# Data Transformation and Preprocessing on Retail Sales Dataset

This Jupyter notebook outlines the processes used to preprocess and transform the retail sales dataset. The transformations are designed to clean, enrich, and reorganize the data for subsequent analysis.

The notebook covers essential preprocessing tasks, including handling missing values, enriching the data with random assignments, generating new features based on business logic, and structuring the data for analysis.

---

### Overview of Key Points:
- **Customer Demographics:** We create and merge `customer demographic data`, including assigning random `cities` and `provinces`.
- **Seasonal Classification:** Transactions are categorized by `season` based on the date of the transaction.
- **Customer Type:** Customers are labeled as `New` or `Returning` based on their transaction history.
- **Data Enrichment:** Additional columns like `Total Spend`, `Product ID`, and `Season` are generated to provide more useful insights into the dataset.
- **Data Saving:** The transformed data is saved into a new CSV file for future use.

---

### 1. Load and Display Initial Dataset

We begin by loading the retail sales dataset using `pandas.read_csv()`. The dataset is stored in the DataFrame `df` and displayed for inspection.


In [1]:
import pandas as pd
df = pd.read_csv('./raw/retail_sales_dataset.csv')
df

,Transaction ID,Date,Customer ID,Gender,Age,Product Category,Quantity,Price per Unit,Total Amount
0,1,2023-11-24,CUST001,Male,34,Beauty,3,50,150
1,2,2023-02-27,CUST002,Female,26,Clothing,2,500,1000
2,3,2023-01-13,CUST003,Male,50,Electronics,1,30,30
3,4,2023-05-21,CUST004,Male,37,Clothing,1,500,500
4,5,2023-05-06,CUST005,Male,30,Beauty,2,50,100
...,...,...,...,...,...,...,...,...,...
995,996,2023-05-16,CUST996,Male,62,Clothing,1,50,50
996,997,2023-11-17,CUST997,Male,52,Beauty,3,30,90
997,998,2023-10-29,CUST998,Female,23,Beauty,4,25,100
998,999,2023-12-05,CUST999,Female,36,Electronics,3,50,150


### 2. Create Customer Demographic Data

The first 50 rows of the dataset are extracted to create a new DataFrame (`customers_df`) containing only `Customer ID`, `Gender`, and `Age` columns. This subset will be used for demographic enrichment.

In [2]:
customers_df = df.loc[:49,['Customer ID','Gender','Age']]
customers_df

,Customer ID,Gender,Age
0,CUST001,Male,34
1,CUST002,Female,26
2,CUST003,Male,50
3,CUST004,Male,37
4,CUST005,Male,30
5,CUST006,Female,45
6,CUST007,Male,46
7,CUST008,Male,30
8,CUST009,Male,63
9,CUST010,Female,52


### 3. Remove Customer Demographics from Main Data

We drop the `Customer ID`, `Gender`, and `Age` columns from the main dataset (df) to avoid redundancy after enriching the data.

In [3]:
df.drop(columns=['Customer ID','Gender','Age'],inplace=True)
df

,Transaction ID,Date,Product Category,Quantity,Price per Unit,Total Amount
0,1,2023-11-24,Beauty,3,50,150
1,2,2023-02-27,Clothing,2,500,1000
2,3,2023-01-13,Electronics,1,30,30
3,4,2023-05-21,Clothing,1,500,500
4,5,2023-05-06,Beauty,2,50,100
...,...,...,...,...,...,...
995,996,2023-05-16,Clothing,1,50,50
996,997,2023-11-17,Beauty,3,30,90
997,998,2023-10-29,Beauty,4,25,100
998,999,2023-12-05,Electronics,3,50,150


### 4. Create Province-City Mapping for generating geo-data

A dictionary (`province_city_dict`) is created to map Canadian provinces to their respective cities. This will be used later to assign random cities and provinces to customers.

In [4]:
province_city_dict = {
    'Alberta': ['Calgary', 'Edmonton', 'Red Deer', 'Lethbridge'],
    'British Columbia': ['Vancouver', 'Victoria', 'Surrey', 'Burnaby'],
    'Manitoba': ['Winnipeg', 'Brandon', 'Steinbach', 'Thompson'],
    'New Brunswick': ['Fredericton', 'Moncton', 'Saint John'],
    'Newfoundland and Labrador': ['St. John\'s', 'Mount Pearl', 'Corner Brook'],
    'Nova Scotia': ['Halifax', 'Sydney', 'Dartmouth'],
    'Ontario': ['Toronto', 'Ottawa', 'Hamilton', 'Kitchener'],
    'Prince Edward Island': ['Charlottetown', 'Summerside'],
    'Quebec': ['Montreal', 'Quebec City', 'Laval', 'Gatineau'],
    'Saskatchewan': ['Saskatoon', 'Regina', 'Moose Jaw']
}

province_city_dict

{'Alberta': ['Calgary', 'Edmonton', 'Red Deer', 'Lethbridge'],
 'British Columbia': ['Vancouver', 'Victoria', 'Surrey', 'Burnaby'],
 'Manitoba': ['Winnipeg', 'Brandon', 'Steinbach', 'Thompson'],
 'New Brunswick': ['Fredericton', 'Moncton', 'Saint John'],
 'Newfoundland and Labrador': ["St. John's", 'Mount Pearl', 'Corner Brook'],
 'Nova Scotia': ['Halifax', 'Sydney', 'Dartmouth'],
 'Ontario': ['Toronto', 'Ottawa', 'Hamilton', 'Kitchener'],
 'Prince Edward Island': ['Charlottetown', 'Summerside'],
 'Quebec': ['Montreal', 'Quebec City', 'Laval', 'Gatineau'],
 'Saskatchewan': ['Saskatoon', 'Regina', 'Moose Jaw']}

### 5. Assign Random City and Province to Customers

Using the `province_city_dict`, we randomly assign a province and a city to each customer. Additionally, a `Country` column is added with a default value of `Canada`. This information is then added to `customers_df`.

In [5]:
import numpy as np
np.random.seed(42)

# Function to assign random city and province
def assign_city_and_province():
    province = np.random.choice(list(province_city_dict.keys()))  # Randomly pick a province
    city = np.random.choice(province_city_dict[province])  # Randomly pick a city from the selected province
    return province, city

# Apply the function to add 'Province' and 'City' columns to the DataFrame

for index, row in customers_df.iterrows():
    province, city = assign_city_and_province()
    customers_df.at[index, 'City'] = city
    customers_df.at[index, 'Province'] = province
   

# Add 'Country' column with all values set to 'Canada'
customers_df['Country'] = 'Canada'
customers_df

,Customer ID,Gender,Age,City,Province,Country
0,CUST001,Male,34,Kitchener,Ontario,Canada
1,CUST002,Female,26,Charlottetown,Prince Edward Island,Canada
2,CUST003,Male,50,Corner Brook,Newfoundland and Labrador,Canada
3,CUST004,Male,37,Moose Jaw,Saskatchewan,Canada
4,CUST005,Male,30,Hamilton,Ontario,Canada
5,CUST006,Female,45,Charlottetown,Prince Edward Island,Canada
6,CUST007,Male,46,Saint John,New Brunswick,Canada
7,CUST008,Male,30,Halifax,Nova Scotia,Canada
8,CUST009,Male,63,Burnaby,British Columbia,Canada
9,CUST010,Female,52,Sydney,Nova Scotia,Canada


### 6. Merge Customer Demographics with Main Data

We randomly assign a `Customer ID` from the `customers_df` to each row in the main dataset (`df`). The customer demographics are then merged with the main dataset based on the `Customer ID`.

In [6]:
# Randomly assign 'Customer ID' from customers_df to df
df['Customer ID'] = np.random.choice(customers_df['Customer ID'], size=len(df))

# Merge customer demographics into df
df = df.merge(customers_df[['Customer ID', 'Gender', 'Age', 'City', 'Province', 'Country']], on='Customer ID', how='left')

df

,Transaction ID,Date,Product Category,Quantity,Price per Unit,Total Amount,Customer ID,Gender,Age,City,Province,Country
0,1,2023-11-24,Beauty,3,50,150,CUST008,Male,30,Halifax,Nova Scotia,Canada
1,2,2023-02-27,Clothing,2,500,1000,CUST035,Female,58,Victoria,British Columbia,Canada
2,3,2023-01-13,Electronics,1,30,30,CUST035,Female,58,Victoria,British Columbia,Canada
3,4,2023-05-21,Clothing,1,500,500,CUST033,Female,50,Sydney,Nova Scotia,Canada
4,5,2023-05-06,Beauty,2,50,100,CUST005,Male,30,Hamilton,Ontario,Canada
...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,2023-05-16,Clothing,1,50,50,CUST014,Male,64,Moose Jaw,Saskatchewan,Canada
996,997,2023-11-17,Beauty,3,30,90,CUST030,Female,39,Lethbridge,Alberta,Canada
997,998,2023-10-29,Beauty,4,25,100,CUST035,Female,58,Victoria,British Columbia,Canada
998,999,2023-12-05,Electronics,3,50,150,CUST021,Female,50,Burnaby,British Columbia,Canada


### 7. Add Season Information Based on Date
We convert the `Date` column to a datetime format and then create a new column `Season` to categorize transactions into one of four seasons (Spring, Summer, Fall, Winter) based on the date.

In [7]:
df['Date'] = pd.to_datetime(df['Date'])

# Define a function to determine the season based on the date
def get_season(date):
    # Get the month and day from the date
    month_day = date.month * 100 + date.day  # Convert month and day into a number (e.g., March 21 -> 321)

    # Determine season based on the month and day
    if 321 <= month_day <= 620:
        return 'Spring'
    elif 621 <= month_day <= 922:
        return 'Summer'
    elif 923 <= month_day <= 1220:
        return 'Fall'
    else:  # Covers both Winter cases (Dec 21 - Dec 31 and Jan 1 - Mar 20)
        return 'Winter'

# Apply the function to create the 'Season' column
df['Season'] = df['Date'].apply(get_season)

df

,Transaction ID,Date,Product Category,Quantity,Price per Unit,Total Amount,Customer ID,Gender,Age,City,Province,Country,Season
0,1,2023-11-24,Beauty,3,50,150,CUST008,Male,30,Halifax,Nova Scotia,Canada,Fall
1,2,2023-02-27,Clothing,2,500,1000,CUST035,Female,58,Victoria,British Columbia,Canada,Winter
2,3,2023-01-13,Electronics,1,30,30,CUST035,Female,58,Victoria,British Columbia,Canada,Winter
3,4,2023-05-21,Clothing,1,500,500,CUST033,Female,50,Sydney,Nova Scotia,Canada,Spring
4,5,2023-05-06,Beauty,2,50,100,CUST005,Male,30,Hamilton,Ontario,Canada,Spring
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,2023-05-16,Clothing,1,50,50,CUST014,Male,64,Moose Jaw,Saskatchewan,Canada,Spring
996,997,2023-11-17,Beauty,3,30,90,CUST030,Female,39,Lethbridge,Alberta,Canada,Fall
997,998,2023-10-29,Beauty,4,25,100,CUST035,Female,58,Victoria,British Columbia,Canada,Fall
998,999,2023-12-05,Electronics,3,50,150,CUST021,Female,50,Burnaby,British Columbia,Canada,Fall


### 8. Assign Random Discount Flag

A random `True` or `False` value is assigned to the `is_discount` column, representing whether a transaction had a discount.

In [8]:
df['is_discount'] = np.random.choice([True, False], size=len(df))
df

,Transaction ID,Date,Product Category,Quantity,Price per Unit,Total Amount,Customer ID,Gender,Age,City,Province,Country,Season,is_discount
0,1,2023-11-24,Beauty,3,50,150,CUST008,Male,30,Halifax,Nova Scotia,Canada,Fall,True
1,2,2023-02-27,Clothing,2,500,1000,CUST035,Female,58,Victoria,British Columbia,Canada,Winter,True
2,3,2023-01-13,Electronics,1,30,30,CUST035,Female,58,Victoria,British Columbia,Canada,Winter,True
3,4,2023-05-21,Clothing,1,500,500,CUST033,Female,50,Sydney,Nova Scotia,Canada,Spring,False
4,5,2023-05-06,Beauty,2,50,100,CUST005,Male,30,Hamilton,Ontario,Canada,Spring,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,2023-05-16,Clothing,1,50,50,CUST014,Male,64,Moose Jaw,Saskatchewan,Canada,Spring,True
996,997,2023-11-17,Beauty,3,30,90,CUST030,Female,39,Lethbridge,Alberta,Canada,Fall,True
997,998,2023-10-29,Beauty,4,25,100,CUST035,Female,58,Victoria,British Columbia,Canada,Fall,False
998,999,2023-12-05,Electronics,3,50,150,CUST021,Female,50,Burnaby,British Columbia,Canada,Fall,False


### 9. Sort Data by Date

The dataset is sorted in ascending order by the `Date` column to ensure that the transactions are `ordered chronologically`.

In [9]:
df = df.sort_values(by='Date', ascending=True)
df

,Transaction ID,Date,Product Category,Quantity,Price per Unit,Total Amount,Customer ID,Gender,Age,City,Province,Country,Season,is_discount
521,522,2023-01-01,Beauty,3,500,1500,CUST045,Female,55,Montreal,Quebec,Canada,Winter,True
179,180,2023-01-01,Clothing,3,300,900,CUST041,Male,34,Charlottetown,Prince Edward Island,Canada,Winter,True
558,559,2023-01-01,Clothing,4,300,1200,CUST022,Male,18,Gatineau,Quebec,Canada,Winter,True
302,303,2023-01-02,Electronics,3,30,90,CUST007,Male,46,Saint John,New Brunswick,Canada,Winter,True
978,979,2023-01-02,Beauty,1,25,25,CUST044,Female,22,Gatineau,Quebec,Canada,Winter,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232,233,2023-12-29,Beauty,2,300,600,CUST027,Female,38,Kitchener,Ontario,Canada,Winter,True
804,805,2023-12-29,Beauty,3,500,1500,CUST046,Female,20,Calgary,Alberta,Canada,Winter,False
856,857,2023-12-31,Electronics,2,25,50,CUST017,Female,27,Laval,Quebec,Canada,Winter,True
210,211,2024-01-01,Beauty,3,500,1500,CUST033,Female,50,Sydney,Nova Scotia,Canada,Winter,False


### 10. Assign Transaction IDs

A `Transaction` ID is assigned sequentially to each transaction, starting from 1.

In [10]:
df['Transaction ID'] = range(1, len(df) + 1)
df

,Transaction ID,Date,Product Category,Quantity,Price per Unit,Total Amount,Customer ID,Gender,Age,City,Province,Country,Season,is_discount
521,1,2023-01-01,Beauty,3,500,1500,CUST045,Female,55,Montreal,Quebec,Canada,Winter,True
179,2,2023-01-01,Clothing,3,300,900,CUST041,Male,34,Charlottetown,Prince Edward Island,Canada,Winter,True
558,3,2023-01-01,Clothing,4,300,1200,CUST022,Male,18,Gatineau,Quebec,Canada,Winter,True
302,4,2023-01-02,Electronics,3,30,90,CUST007,Male,46,Saint John,New Brunswick,Canada,Winter,True
978,5,2023-01-02,Beauty,1,25,25,CUST044,Female,22,Gatineau,Quebec,Canada,Winter,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232,996,2023-12-29,Beauty,2,300,600,CUST027,Female,38,Kitchener,Ontario,Canada,Winter,True
804,997,2023-12-29,Beauty,3,500,1500,CUST046,Female,20,Calgary,Alberta,Canada,Winter,False
856,998,2023-12-31,Electronics,2,25,50,CUST017,Female,27,Laval,Quebec,Canada,Winter,True
210,999,2024-01-01,Beauty,3,500,1500,CUST033,Female,50,Sydney,Nova Scotia,Canada,Winter,False


### 11. Label Customers as 'New' or 'Returning'

A `Customer Type` column is added, labeling customers as either 'New' or 'Returning' based on whether they have made a prior transaction.

In [11]:
new_customers = set()

# Function to label customers as 'New' or 'Returning' in a cumulative manner
def get_customer_type(row):
    if row['Customer ID'] not in new_customers:
        new_customers.add(row['Customer ID'])
        return 'New'
    else:
        return 'Returning'

# Apply the function to the DataFrame to create the 'Customer Type' column
df['Customer Type'] = df.apply(get_customer_type, axis=1)

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, 521 to 649
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Transaction ID    1000 non-null   int64         
 1   Date              1000 non-null   datetime64[ns]
 2   Product Category  1000 non-null   object        
 3   Quantity          1000 non-null   int64         
 4   Price per Unit    1000 non-null   int64         
 5   Total Amount      1000 non-null   int64         
 6   Customer ID       1000 non-null   object        
 7   Gender            1000 non-null   object        
 8   Age               1000 non-null   int64         
 9   City              1000 non-null   object        
 10  Province          1000 non-null   object        
 11  Country           1000 non-null   object        
 12  Season            1000 non-null   object        
 13  is_discount       1000 non-null   bool          
 14  Customer Type     1000 non-n

### 12. Calculate Total Spend per Customer

The total amount spent by each customer (`Total Spend`) is calculated and added to the dataset.

In [12]:
total_spend = df.groupby('Customer ID')['Total Amount'].sum().reset_index()

# Rename the 'Total Amount' column to 'Total Spend'
total_spend.rename(columns={'Total Amount': 'Total Spend'}, inplace=True)

# Merge the total spend back into the original dataframe
df = df.merge(total_spend, on='Customer ID', how='left')

print(df[['Customer ID', 'Total Spend']].head())
df

  Customer ID  Total Spend
0     CUST045        10525
1     CUST041         7650
2     CUST022        11785
3     CUST007         4910
4     CUST044         8150


,Transaction ID,Date,Product Category,Quantity,Price per Unit,Total Amount,Customer ID,Gender,Age,City,Province,Country,Season,is_discount,Customer Type,Total Spend
0,1,2023-01-01,Beauty,3,500,1500,CUST045,Female,55,Montreal,Quebec,Canada,Winter,True,New,10525
1,2,2023-01-01,Clothing,3,300,900,CUST041,Male,34,Charlottetown,Prince Edward Island,Canada,Winter,True,New,7650
2,3,2023-01-01,Clothing,4,300,1200,CUST022,Male,18,Gatineau,Quebec,Canada,Winter,True,New,11785
3,4,2023-01-02,Electronics,3,30,90,CUST007,Male,46,Saint John,New Brunswick,Canada,Winter,True,New,4910
4,5,2023-01-02,Beauty,1,25,25,CUST044,Female,22,Gatineau,Quebec,Canada,Winter,True,New,8150
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,2023-12-29,Beauty,2,300,600,CUST027,Female,38,Kitchener,Ontario,Canada,Winter,True,Returning,4250
996,997,2023-12-29,Beauty,3,500,1500,CUST046,Female,20,Calgary,Alberta,Canada,Winter,False,Returning,4380
997,998,2023-12-31,Electronics,2,25,50,CUST017,Female,27,Laval,Quebec,Canada,Winter,True,Returning,9755
998,999,2024-01-01,Beauty,3,500,1500,CUST033,Female,50,Sydney,Nova Scotia,Canada,Winter,False,Returning,20180


### 13. Assign Product ID

A unique Product ID is generated for each combination of `Product Category` and `Price per Unit`.

In [13]:
df['Product ID'] = df.groupby(['Product Category', 'Price per Unit']).ngroup() + 1

print(df[['Transaction ID', 'Product Category', 'Price per Unit', 'Product ID']])

     Transaction ID Product Category  Price per Unit  Product ID
0                 1           Beauty             500           5
1                 2         Clothing             300           9
2                 3         Clothing             300           9
3                 4      Electronics              30          12
4                 5           Beauty              25           1
..              ...              ...             ...         ...
995             996           Beauty             300           4
996             997           Beauty             500           5
997             998      Electronics              25          11
998             999           Beauty             500           5
999            1000      Electronics              30          12

[1000 rows x 4 columns]


### 14. Save Processed Data

Finally, the transformed dataset is saved to a new CSV file (`retail_sales_dataset.csv`) for further analysis or reporting.


In [14]:
df.to_csv('./processed/retail_sales_dataset.csv', index=False)

### Optional verifications

- Total number of unique customers in the dataset.

In [15]:
customer_transactions = df.groupby('Customer ID').size()
customer_transactions

Customer ID
CUST001    25
CUST002    21
CUST003    22
CUST004    16
CUST005    26
CUST006    22
CUST007    11
CUST008    20
CUST009    18
CUST010    17
CUST011    20
CUST012    22
CUST013    19
CUST014    17
CUST015    15
CUST016    19
CUST017    21
CUST018    13
CUST019    19
CUST020    18
CUST021    16
CUST022    21
CUST023    22
CUST024    20
CUST025    20
CUST026    30
CUST027    15
CUST028    28
CUST029    17
CUST030    22
CUST031    16
CUST032    25
CUST033    29
CUST034    18
CUST035    31
CUST036    20
CUST037    29
CUST038    16
CUST039    22
CUST040    16
CUST041    18
CUST042    15
CUST043    11
CUST044    18
CUST045    22
CUST046    13
CUST047    23
CUST048    21
CUST049    29
CUST050    16
dtype: int64

- Distinct Product IDs Count

In [16]:
distinct_product_ids = df['Product ID'].drop_duplicates()
distinct_product_ids.size

15